In [ ]:
import torch.nn as nn
import torchvision
import torch.utils.data
import torch.optim as optim
from models import BasicParkingNet

In [ ]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(),
        torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        torchvision.transforms.Resize((40, 60)),
        torchvision.transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=32,
    shuffle=True
)

In [ ]:
# Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BasicParkingNet().to(device).train()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()
num_episodes = 100

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../models/basic_parking_net.pth")
        print("\tSaving!")
